In [1]:
import polars as pl
from op_datasets.processing.execute import reader, BatchInput
from op_datasets.logic.transforms.daily_address_summary import (
    daily_address_summary,
    CONDITIONS,
)
from op_coreutils.logger import structlog
from op_coreutils.bigquery.write import overwrite_partitions

from datetime import datetime
import os

os.environ["OPLABS_ENV"] = "PROD"


log = structlog.get_logger()

2024-10-17 11:30:14 [info     ] Loaded env var: OP_ANALYTICS_VAULT


In [2]:
inputbatch: BatchInput = None
for _inputbatch in reader(
    chain="op", block_spec="126071012:+500", source_spec="goldsky"
):
    inputbatch = _inputbatch
    break

dataframes = inputbatch.dataframes

2024-10-17 11:30:15 [info     ] Loaded 5 items into vault.    
2024-10-17 11:30:16 [info     ] Initialized Clickhouse client.
2024-10-17 11:30:16 [info     ] Query success: blocks in 1.26s 2000 rows
2024-10-17 11:30:18 [info     ] Query success: transactions in 2.57s 43972 rows
2024-10-17 11:30:18 [info     ] Query success: logs in 2.76s 187138 rows


In [3]:
dataframes

{'blocks': shape: (1_012, 24)
 ┌───────┬─────────┬──────────┬────────────┬───┬──────────┬───────────┬────────────┬────────────────┐
 │ chain ┆ network ┆ chain_id ┆ dt         ┆ … ┆ gas_used ┆ gas_limit ┆ extra_data ┆ transaction_co │
 │ ---   ┆ ---     ┆ ---      ┆ ---        ┆   ┆ ---      ┆ ---       ┆ ---        ┆ unt            │
 │ str   ┆ str     ┆ i32      ┆ str        ┆   ┆ i64      ┆ i64       ┆ str        ┆ ---            │
 │       ┆         ┆          ┆            ┆   ┆          ┆           ┆            ┆ i64            │
 ╞═══════╪═════════╪══════════╪════════════╪═══╪══════════╪═══════════╪════════════╪════════════════╡
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 1779886  ┆ 30000000  ┆ 0x         ┆ 14             │
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 4285527  ┆ 30000000  ┆ 0x         ┆ 27             │
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 6187833  ┆ 30000000  ┆ 0x         ┆ 30             │
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 516

In [4]:
# check if our blocks are in the range we want

log.info(f"Min block number: {dataframes['blocks']['number'].min()}")
log.info(f"Max block number: {dataframes['blocks']['number'].max()}")
log.info(
    f"Min block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].min())}"
)
log.info(
    f"Max block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].max())}"
)

2024-10-17 11:30:18 [info     ] Min block number: 126070000   
2024-10-17 11:30:18 [info     ] Max block number: 126071011   
2024-10-17 11:30:18 [info     ] Min block number: 2024-09-30 19:26:17
2024-10-17 11:30:18 [info     ] Max block number: 2024-09-30 19:59:59


In [5]:
result = daily_address_summary(
    dataframes["transactions"],
    conditions=CONDITIONS,
    address="from_address",
    chain_id="chain_id",
    chain="chain",
    dt="dt",
)

In [6]:
# list(result.head()["hash"])
result.head()

from_address,chain_id,chain,dt,total_txs,total_txs_success,total_txs_fail,total_blocks,total_blocks_success,total_blocks_fail,min_block_number,max_block_number,block_interval_active,min_nonce,max_nonce,nonce_interval_active,total_l2_gas_used,total_l2_gas_used_success,total_l2_gas_used_fail,total_l1_gas_used,total_l1_gas_used_success,total_l1_gas_used_fail,total_gas_fees,total_gas_fees_success,total_gas_fees_fail,l2_contrib_gas_fees,l1_contrib_gas_fees,l1_blobgas_contrib_gas_fees,l1_l1gas_contrib_gas_fees,l2_contrib_gas_fees_base_fee,l2_contrib_gas_fees_priority_fee,avg_l2_gas_price_gwei,avg_l2_base_fee_gwei,avg_l2_priority_fee_gwei,avg_l1_gas_price_gwei,avg_l1_blob_base_fee_gwei,min_block_timestamp,max_block_timestamp,time_interval_active,unique_hours_active,perc_daily_hours_active,num_to_addresses,num_to_addresses_success,num_to_addresses_fail
str,i32,str,str,u32,u32,u32,u32,u32,u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,"decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,17]","decimal[*,17]","decimal[*,17]","decimal[*,17]","decimal[*,17]",datetime[μs],datetime[μs],duration[μs],u32,f64,u32,u32,u32
"""0x90ed778edda7ea75f9b331a138a5…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126070736,126070736,1,4175,4175,1,51532,51532,0,1600,1600,0,0.0000052812459416470000,0.0000052812459416470000,0.0000000000000000000000,0.0000052169436926360000,0.0000000643022490110000,0.0000000016227408000000,0.0000000000000000000000,0.0000000637436926360000,0.0000051532000000000000,0.10123697300000000,0.00123697300000000,0.10000000000000000,7.68871352000000000,0.00000000100000000,2024-09-30 23:50:49,2024-09-30 23:50:49,0µs,1,0.041667,1,1,0
"""0xba3534f139f02b3dc358175858eb…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126070458,126070458,1,2,2,1,34750,34750,0,1600,1600,0,0.0000004399850244850000,0.0000004399850244850000,0.0000000000000000000000,0.0000003844624977500000,0.0000000555225267350000,0.0000000016227408000000,0.0000000000000000000000,0.0000000369624977500000,0.0000003475000000000000,0.01106366900000000,0.00106366900000000,0.01000000000000000,6.63890934500000000,0.00000000100000000,2024-09-30 23:41:33,2024-09-30 23:41:33,0µs,1,0.041667,1,1,0
"""0xbdccb78e31a28e84235491e8a2d7…",10,"""op""","""2024-09-30""",9,9,0,9,9,0,126070076,126070742,667,40756,40764,9,2197735,2197735,0,14400,14400,0,0.0000104140830998830000,0.0000104140830998830000,0.0000000000000000000000,0.0000099395688969620000,0.0000004745142029210000,0.0000000146046672000000,0.0000000000000000000000,0.0000021127629035180000,0.0000078268059934440000,0.00452264212790077,0.00096133651396460,0.00356130561393616,6.30426155344444444,0.00000000100000000,2024-09-30 23:28:49,2024-09-30 23:51:01,22m 12s,1,0.041667,1,1,0
"""0xc3be5fc83d407f5810b0b0cd4a82…",10,"""op""","""2024-09-30""",2,2,0,2,2,0,126070114,126070980,867,309,310,2,59240,59240,0,3200,3200,0,0.0000002975833542980000,0.0000002975833542980000,0.0000000000000000000000,0.0000001817146716800000,0.0000001158686826180000,0.0000000032454816000000,0.0000000000000000000000,0.0000001817146716800000,0.0000000000000000000000,0.00306743200000000,0.00306743200000000,0.00000000000000000,6.92729352500000000,0.00000000100000000,2024-09-30 23:30:05,2024-09-30 23:58:57,28m 52s,1,0.041667,1,1,0
"""0x6a135231ecf380bf85bc9c302c18…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126070323,126070323,1,0,0,1,21000,21000,0,1600,1600,0,0.0000000984047167430000,0.0000000984047167430000,0.0000000000000000000000,0.0000000427536480000000,0.0000000556510687430000,0.0000000016227408000000,0.0000000000000000000000,0.0000000427536480000000,0.0000000000000000000000,0.00203588800000000,0.00203588800000000,0.00000000000000000,6.65427930000000000,0.00000000100000000,2024-09-30 23:37:03,2024-09-30 23:37:03,0µs,1,0.041667,1,1,0


In [7]:
overwrite_partitions(result, "temp", "daily_address_summary", expiration_days=999)

2024-10-17 11:30:18 [info     ] Writing 1 partitions to BQ [2024-09-30 00:00:00 ... 2024-09-30 00:00:00]
2024-10-17 11:30:18 [info     ] DRYRUN OVERWRITE PARTITION: Wrote 2.3Krows 290.7KB to BQ temp.daily_address_summary
